In [1]:
#使用自己的IBMQ帳號
import qiskit as Q
#輸入API
Q.IBMQ.enable_account('9eacd7290c36819ccce35fe32140ae3c5ef276f4b950dc2d97564a0c4399147530fcc08247fcbd0507c17a83f6767d48be1d6184aa3a0118103893ae2daba175')

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
from qiskit import *
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.providers.aer import QasmSimulator
from math import pi, sqrt, sin, cos
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os,sys
from pandas import *
import pandas as pd

In [3]:
#使用的simulator
#simulator = QasmSimulator()
#GHZ-states得知之最大角度帶入
params= 1/4*pi
#定義GHZ的quantum circuit
def vqe_ansatz_1(params):
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    ansatz = QuantumCircuit(q,c)
    #GHZ-like state
    ansatz.h(q[0])
    ansatz.cx(q[0],q[1])
    ansatz.rz(params,q[0])
    return ansatz

In [4]:
sample_shots = 1024
def expval(params):
    def partial(params):
        # YX的quantum circuit
        vqe= vqe_ansatz_1(params) #call ansatz
        vqe.sdg(0)
        vqe.h(0)
        vqe.h(1)
        vqe.measure([0],[0])
        vqe.measure([1],[1])

        # Compile and execute
        # use real quantum computer
        provider = Q.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
        qcomp = provider.get_backend('ibmq_manila') #called real quantum computer
        # calculation
        job_qasm = execute(vqe, qcomp, initial_layout = [3,4], shots=sample_shots)
        result_sim = job_qasm.result().get_counts()
        # Calculate expectation
        P_0=(result_sim.get('00')/sample_shots)+(result_sim.get('11')/sample_shots)
        P_1=(result_sim.get('01')/sample_shots)+(result_sim.get('10')/sample_shots)
        YX = P_0-P_1

        # YX的quantum circuit
        vqe= vqe_ansatz_1(params) #call ansatz
        vqe.sdg(1)
        vqe.h(0)
        vqe.h(1)
        vqe.measure([0],[0])
        vqe.measure([1],[1])

        # Compile and execute
        # use real quantum computer
        provider = Q.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
        qcomp = provider.get_backend('ibmq_manila') #called real quantum computer
        # calculation
        job_qasm = execute(vqe, qcomp, initial_layout = [3,4], shots=sample_shots)
        result_sim = job_qasm.result().get_counts()
        # Calculate expectation
        P_0=(result_sim.get('00')/sample_shots)+(result_sim.get('11')/sample_shots)
        P_1=(result_sim.get('01')/sample_shots)+(result_sim.get('10')/sample_shots)
        XY = P_0-P_1
        
        # XX的quantum circuit
        vqe= vqe_ansatz_1(params)
        vqe.h(0)
        vqe.h(1)
        vqe.measure([0],[0])
        vqe.measure([1],[1])

        # Compile and execute
        # use real quantum computer
        provider = Q.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
        qcomp = provider.get_backend('ibmq_manila') #called real quantum computer
        # calculation
        job_qasm = execute(vqe, qcomp,initial_layout = [3,4], shots=sample_shots)
        result_sim = job_qasm.result().get_counts()
        # Calculate expectation
        P_0=(result_sim.get('00')/sample_shots)+(result_sim.get('11')/sample_shots)
        P_1=(result_sim.get('01')/sample_shots)+(result_sim.get('10')/sample_shots)
        XX = P_0-P_1
    
        # YY的quantum circuit
        vqe= vqe_ansatz_1(params)
        vqe.sdg(0)
        vqe.sdg(1)
        vqe.h(0)
        vqe.h(1)
        vqe.measure([0],[0])
        vqe.measure([1],[1])
    
        # Compile and execute
        # use real quantum computer
        provider = Q.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
        qcomp = provider.get_backend('ibmq_manila') #called real quantum computer
        # calculation
        job_qasm = execute(vqe, qcomp, initial_layout = [3,4], shots=sample_shots)
        result_sim = job_qasm.result().get_counts()
        # Calculate expectation
        P_0=(result_sim.get('00')/sample_shots)+(result_sim.get('11')/sample_shots)
        P_1=(result_sim.get('01')/sample_shots)+(result_sim.get('10')/sample_shots)
        YY = P_0-P_1
        return [YX, XY, XX, YY]
    Y = partial(params) #叫出結果
    M2 = Y[2] + Y[0]+ Y[1] - Y[3] #針對結果計算
    Mb2 = Y[3] + Y[1] + Y[0] - Y[2]
    M2LR = 2**(2/2)
    #normalized
    M2ba = M2/M2LR 
    Mb2ba = Mb2/M2LR
    print (M2)
    return [M2ba, Mb2ba]
X= expval(params)
print (X)

2.478515625
[1.2392578125, -0.0146484375]


In [5]:
#計算以上quantum circuit運行一次所需時間
import time
t0 = time.time()
time.sleep(1)
print('time cost: {}'.format(time.time() - t0))

time cost: 1.0071797370910645


In [6]:
num = 5 #GHZ quantum circuit運轉10次
#將結果列進list中
M2_list = []
Mb2_list = []
for i in range (num):
    X= expval(params)
    M2_list.append(X[0])
    Mb2_list.append(X[1])
    print (X)

2.52734375
[1.263671875, 0.02734375]
2.51953125
[1.259765625, -0.033203125]
2.626953125
[1.3134765625, -0.0029296875]
2.576171875
[1.2880859375, -0.0126953125]
2.541015625
[1.2705078125, -0.0107421875]


In [7]:
#將list中的結果以excel匯出
dict = {'M2': M2_list, 'Mb2': Mb2_list}
df = pd.DataFrame(dict)
df.index = df.index+1
df.to_excel("output_2qubit.xlsx")